In [22]:
import ee
import geopandas as gpd
import pandas as pd
import pandas as pd
import json
import configparser
import time

In [23]:
ee.Authenticate()

True

In [24]:
ee.Initialize()

In [67]:
config = configparser.ConfigParser()
config.read(r"./config.ini")
start_yy = int(config.get("myvars", "start_yy"))
end_yy = int(config.get("myvars", "end_yy"))+1
start_mm = int(config.get("myvars", "start_mm"))
end_mm = int(config.get("myvars", "end_mm"))+1

ee_collection = config.get("myvars", "ee_col")
path = config.get("myvars", "path")
band_ee = config.get("myvars", "band_ee")

folder = config.get("myvars", "folder")

In [68]:
list_anuva = []
data = ee.ImageCollection(ee_collection);

data_band = data.select(band_ee)

for yy in range(start_yy,end_yy):
    #yy_count = 1
    
    for mm in range (start_mm,end_mm):
        print(mm)
        
        if mm in [1, 3, 5, 7, 8, 10, 12]:
            days = 31
        elif mm in [4, 6, 9, 11]:
            days = 30
        else:
            days = 28  # Assuming February has 28 days (non-leap year)
        print(days)
        for dd in range(1,days+1):            
    
            if(dd==days):                          
                s_date = str(yy) + '-' + str(mm) + '-' + str(dd)                
                
                if(mm==12):
                    e_date = str(yy+1) + '-' + str((1)) + '-' + str(1)
                else:
                    
                    if(start_yy == end_yy):
                        e_date = str(yy+1) + '-' + str((1)) + '-' + str(1)
                    else:
                        e_date = str(yy) + '-' + str((mm+1)) + '-' + str(1)
                        print(e_date)
            else:
                s_date = str(yy) + '-' + str(mm) + '-' + str(dd)
                e_date = str(yy) + '-' + str(mm) + '-' + str(dd+1)
            
            dataset = data_band.filterDate(s_date, e_date)

            total = dataset.reduce(ee.Reducer.mean())

            geom = gpd.read_file(path)

            js = json.loads(geom.to_json())
            shpz = ee.Geometry(ee.FeatureCollection(js).geometry())

            stats = total.reduceRegion(
              reducer= ee.Reducer.mean(),
              geometry= shpz,
              scale= 5000,
              ).get('precipitation_mean')

            print(s_date + "  Amount = ",stats.getInfo())

            list_anuva.append(stats.getInfo())
        
   # if(yy_count>1):
   #    time.sleep(5*60)
   #   yy_count = yy_count+1
print(list_anuva)

1
31
2023-1-1  Amount =  0
2023-1-2  Amount =  1.1355864068526935
2023-1-3  Amount =  0
2023-1-4  Amount =  0
2023-1-5  Amount =  0
2023-1-6  Amount =  0.0772746558275651
2023-1-7  Amount =  0
2023-1-8  Amount =  0
2023-1-9  Amount =  0
2023-1-10  Amount =  0.216119826149567
2023-1-11  Amount =  0.9713084533195265
2023-1-12  Amount =  0
2023-1-13  Amount =  0
2023-1-14  Amount =  0
2023-1-15  Amount =  0
2023-1-16  Amount =  0
2023-1-17  Amount =  0.16349317148875991
2023-1-18  Amount =  0.30617886652373666
2023-1-19  Amount =  0
2023-1-20  Amount =  0
2023-1-21  Amount =  0.002876787355010011
2023-1-22  Amount =  0
2023-1-23  Amount =  0.12133472252417303
2023-1-24  Amount =  0
2023-1-25  Amount =  0
2023-1-26  Amount =  0
2023-1-27  Amount =  0.0999786701130992
2023-1-28  Amount =  0
2023-1-29  Amount =  1.2690367683903485
2023-1-30  Amount =  1.5634872772383213
2023-2-1
2023-1-31  Amount =  0
2
28
2023-2-1  Amount =  0
2023-2-2  Amount =  0
2023-2-3  Amount =  0
2023-2-4  Amount =  

In [69]:
import pandas
df = pandas.DataFrame(data={"col1": list_anuva})
df.to_csv("./birbhum_2023_DAILY.csv", sep=',',index=False)